In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import datetime
import time
import re


#Filter 1
def fetch_issuers():
    url = 'https://www.mse.mk/en/stats/symbolhistory/KMB'  
    response = requests.get(url)
    if response.status_code != 200:
        print("Failed to retrieve data")
        return None

    soup = BeautifulSoup(response.content, 'html.parser')

    # Find the dropdown menu for issuers
    issuers_dropdown = soup.find('select', {'id': 'Code'})  
    issuers = []

    for option in issuers_dropdown.find_all('option'):
        code = option.get('value')
        name = option.text
        if code and not any(char.isdigit() for char in code):  
            issuers.append({'Issuer Code': code})

    # Convert to DataFrame and return
    return pd.DataFrame(issuers)


#Filter 2
def check_last_date_of_data(issuer_code, df):
        if(df.empty):
            return None
        # Filter the data for the given issuer
        issuer_data = df[df['Issuer Code'] == issuer_code]
        #print(issuer_data)
        
        if not issuer_data.empty:
            # Ensure the 'Date' column is in datetime format
            issuer_data['Date'] = pd.to_datetime(issuer_data['Date'], errors='coerce')
            
            # Get the last available date
            last_date = issuer_data['Date'].max()
            
            if pd.isna(last_date):
                print(f"Invalid date entries found for {issuer_code}.")
            else:
                # Return the date in 'm/d/y' format
                last_date_str = last_date.strftime("%m/%d/%Y")
                #print(f"Last data for {issuer_code} is from {last_date_str}")
                return last_date_str
        else:
            print(f"No data found for {issuer_code}, need to fetch from the last 10 years.")
            return None



#Filter 3
def parse_cells(row, code):
        translation_table = str.maketrans({',': '.',
                                           '.': ','})

        cells = row.find_all('td')
        date = cells[0].text
        last_trade_price = cells[1].text.translate(translation_table)
        max = cells[2].text.translate(translation_table)
        min = cells[3].text.translate(translation_table)
        avg_price = cells[4].text.translate(translation_table)
        chg = cells[5].text.translate(translation_table)
        volume = cells[6].text.translate(translation_table)
        turnover_in_best = cells[7].text.translate(translation_table)
        total_turnover = cells[8].text.translate(translation_table)

        return [code, date, last_trade_price, max, min, avg_price, chg, volume, turnover_in_best, total_turnover]


def fetch_stock_data(issuer_code, start_date, end_date):
    base_url = "https://www.mse.mk/en/stats/symbolhistory"
    url = f"{base_url}/{issuer_code}"

    from_date_str = start_date.strftime("%m.%d.%y")
    to_date_str = end_date.strftime("%m.%d.%y")
    form_data = {
        'FromDate': from_date_str,
        'ToDate': to_date_str,
    }
    print(f"Scraping data from {from_date_str} to {to_date_str} for {issuer_code}")
    response = requests.post(url, data=form_data)
    if response.status_code == 200:
        print("Successfully fetched data")
        soup = BeautifulSoup(response.content, 'html.parser')
        table = soup.find('table', {'id': 'resultsTable'})

        if table is None:
            print("Table not found.")
            return None

        rows = table.find_all('tr')
        data = []
        for row in rows[1:]:
            parsed_row = parse_cells(row, issuer_code)
            if parsed_row:
                data.append(parsed_row)
        return data
    else:
        print("Failed to fetch data")
        return None
    

#Filter 4
def scrape_last_10_years(issuer_code):
    # Set the end date to today's date
    today = datetime.date.today()

    # List to store all data
    all_data = []

    # Start from today and move backward one year at a time
    current_end_date = today

    # Loop to cover each year until 10 years ago
    for _ in range(10):
        # Calculate the start date, one year ago from the current end date
        current_start_date = current_end_date.replace(year=current_end_date.year - 1)

        # If the calculated start date goes beyond the 10-year limit, adjust it
        ten_years_ago = today.replace(year=today.year - 10)
        if current_start_date < ten_years_ago:
            current_start_date = ten_years_ago

        
        
        # Fetch data for the current year range
        year_data = fetch_stock_data(issuer_code, current_start_date, current_end_date)
        if year_data:
            all_data.extend(year_data)
        # Move the end date backward for the next iteration
        current_end_date = current_start_date - datetime.timedelta(days=1)

    # Convert the data into a DataFrame
    df = pd.DataFrame(all_data, columns=["Issuer Code","Date", "Last trade price", "Max", "Min", "Avg.Price", "%chg.", "Volume", "Turnover in BEST in denars", "Total turnover in denars"])
    return df


#Filter 5
def save_issuers_to_excel(df, filename='issuers.xlsx'):
    try:
        # Try to load the existing Excel file for issuers
        with pd.ExcelWriter(filename, engine='openpyxl', mode='a', if_sheet_exists='replace') as writer:
            df.to_excel(writer, sheet_name='Issuers', index=False)
            print(f"Issuers data saved to '{filename}' with 'Issuers' sheet.")
    except FileNotFoundError:
        # If the file doesn't exist, create a new file with the 'Issuers' sheet
        with pd.ExcelWriter(filename, engine='openpyxl') as writer:
            df.to_excel(writer, sheet_name='Issuers', index=False)
            print(f"Excel file '{filename}' created with 'Issuers' sheet.")


def save_stock_data_to_excel(df, filename='stock_data.xlsx'):
    try:
        # Try to load the existing Excel file for stock data
        with pd.ExcelWriter(filename, engine='openpyxl', mode='a', if_sheet_exists='replace') as writer:
            df.to_excel(writer, sheet_name='Stock Data', index=False)
            print(f"Stock data saved to '{filename}' with 'Stock Data' sheet.")
    except FileNotFoundError:
        # If the file doesn't exist, create a new file with the 'Stock Data' sheet
        with pd.ExcelWriter(filename, engine='openpyxl') as writer:
            df.to_excel(writer, sheet_name='Stock Data', index=False)
            print(f"Excel file '{filename}' created with 'Stock Data' sheet.")

#Main Pipeline
def main():
    issuers = fetch_issuers()
    df = pd.DataFrame()

    # Load existing stock data (if any)
    try:
        existing_stock_data = pd.read_excel('stock_data.xlsx')
    except FileNotFoundError:
        existing_stock_data = pd.DataFrame()  # If no existing data, create an empty DataFrame

    for code in issuers['Issuer Code'].values:
        # Check the last available date for the issuer
        last_date = check_last_date_of_data(code,existing_stock_data)
        
        if last_date:
            # If there is data, fetch the new data starting from the next day after the last date
            last_date_obj = datetime.datetime.strptime(last_date, "%m/%d/%Y").date()
            next_start_date = last_date_obj + datetime.timedelta(days=1)
            
            # Fetch the new stock data starting from the next day after the last available date
            new_data = fetch_stock_data(code, next_start_date, datetime.date.today())
            if new_data:
                new_df = pd.DataFrame(new_data, columns=["Issuer Code", "Date", "Last trade price", "Max", "Min", "Avg.Price", "%chg.", "Volume", "Turnover in BEST in denars", "Total turnover in denars"])
                
                # Separate the existing data for the issuer from the entire DataFrame
                existing_issuer_data = existing_stock_data[existing_stock_data['Issuer Code'] == code]
                
                # Prepend new data on top of the existing data for this issuer
                updated_issuer_data = pd.concat([new_df, existing_issuer_data], ignore_index=True)
                
                # Replace the old data for this issuer code with the updated data
                existing_stock_data = existing_stock_data[existing_stock_data['Issuer Code'] != code]
                
                # Add the updated data for the issuer back into the entire DataFrame
                existing_stock_data = pd.concat([existing_stock_data, updated_issuer_data], ignore_index=True)
        else:
            # If no data is found for the issuer, fetch the last 10 years of data
            new_df = scrape_last_10_years(code)
            
            # Add the new data to the DataFrame
            existing_stock_data = pd.concat([existing_stock_data, new_df], ignore_index=True)

    # Save the updated stock data to the Excel file
    save_stock_data_to_excel(existing_stock_data)
    save_issuers_to_excel(issuers)


main()


KeyboardInterrupt: 